#COMP6248 K9DE
Reproduction of baseline Resnet 18/50 results.  First, packages are loaded and functions defined.  Secondly (if you scroll to the very bottom!), runs are performed - a minimal example of how to perform one run is given, plus the code to reproduce the baseline metrics.



#Load packages and team functions

In [1]:
!nvidia-smi
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision
try: 
    import torchbearer
except:
    !pip install torchbearer
device = "cuda:0" if torch.cuda.is_available() else "cpu"
import torchvision
import torchvision.transforms as transforms
import torch 
import torch.nn.functional as F
from torch import nn
import torchbearer
from torchbearer import Trial
from torch import optim
from torchvision.models import resnet18
from torchvision.models import resnet50
from urllib.request import urlopen
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import normalize
from sklearn.svm import OneClassSVM
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, auc
import pandas as pd
import time
from google.colab import drive

Thu May 13 07:48:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
def K9_dataloader(dataset="fMNIST",batch_size=16,shuffle_train_set=False):
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  # COMP6248 Team K9 Density Estimators dataloader
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  # Input: dataset e{"fMNIST","CIFAR10","CIFAR100"}, other arguments are self explanatory
  # Outputs: train and test loaders
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  # Author: Ian
  # Reviewed by: Nic
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  import torchvision
  import torchvision.transforms as transforms
  import torch 
  import torch.nn.functional as F
  from torch import nn
  import torchbearer
  from torchbearer import Trial
  from torch import optim
  from torchvision.models import resnet18
  from torchvision.models import resnet50
  from urllib.request import urlopen
  import matplotlib.pyplot as plt

  #Setup the preprocess function
  if dataset == "fMNIST":
    preprocess_input = transforms.Compose([
                transforms.Resize(256),    #resize 256, then centercrop 224; as per augment.py line 93 and line 97
                transforms.CenterCrop(224),
                transforms.Grayscale(3),   #
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])])   #this is in one of their python scripts, and is standard for ImageNet
  else:
    preprocess_input = transforms.Compose([    #no Grayscale for CIFAR10/CIFAR100
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])])

  #Download the data
  if dataset =="fMNIST":
    train_set = torchvision.datasets.FashionMNIST(root='./data/FashionMNIST',train=True,download=True,transform=preprocess_input)
    test_set = torchvision.datasets.FashionMNIST(root='./data/FashionMNIST',train=False,download=True,transform=preprocess_input)
  if dataset =="CIFAR10":
    train_set = torchvision.datasets.CIFAR10(root='./data/CIFAR10',train=True,download=True,transform=preprocess_input)
    test_set = torchvision.datasets.CIFAR10(root='./data/CIFAR10',train=False,download=True,transform=preprocess_input)
  if dataset =="CIFAR100":
    train_set = torchvision.datasets.CIFAR100(root='./data/CIFAR100',train=True,download=True,transform=preprocess_input)
    test_set = torchvision.datasets.CIFAR100(root='./data/CIFAR100',train=False,download=True,transform=preprocess_input)

  #Setup the loaders
  train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,shuffle=shuffle_train_set)
  test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

  print()
  print(f"train_loader batches: {len(train_loader)}, test_loader batches: {len(test_loader)}, batch_size: {batch_size}")
  print(f"train_shuffle set to {shuffle_train_set}")
  print(f"train_set length: {len(train_set)}, test_set length: {len(test_set)}")
  print(f"train_loader and test_loader ready for {dataset}.")
  print()
  return train_loader, test_loader

In [3]:
def K9_resnet1850_ftex(train_loader, test_loader, mdltype="resnet18", pretrained=False):
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  # COMP6248 Team K9 Density Estimators feature extractor for resnet 18/50
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  # Input: mldtype e{"resnet18","resnet50"}, other arguments self explanatory
  # Outputs: 1D array of class AUCs
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  # Author: Ian
  # Reviewed by: Nic
  ########################################################
  !nvidia-smi
  try:
      import torch
  except:
      from os.path import exists
      from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
      platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
      cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
      accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

      !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision
  try: 
      import torchbearer
  except:
      !pip install torchbearer
  device = "cuda:0" if torch.cuda.is_available() else "cpu"
  ################################################################################
  ############### EXTRACT FEATURES FROM RESNET // based upon https://github.com/ecs-vlc/COMP6248/blob/master/docs/labs/lab6/genfeats.py
  ################################################################################
  import torchvision
  import torchvision.transforms as transforms
  import torch 
  import torch.nn.functional as F
  from torch import nn
  import torchbearer
  from torchbearer import Trial
  from torch import optim
  from torchvision.models import resnet18
  from torchvision.models import resnet50
  from urllib.request import urlopen
  import matplotlib.pyplot as plt

  torch.cuda.empty_cache(); import gc; gc.collect()    #to try to avoid the CUDA out of memory issues...

  if mdltype == "resnet18":
    model = resnet18(pretrained=pretrained)
  if mdltype == "resnet50":
    model = resnet50(pretrained=pretrained)

  feature_extractor_model = nn.Sequential(*list(model.children())[:-2], nn.AdaptiveAvgPool2d((1,1)))
  feature_extractor_model.eval()
  feature_extractor_model = feature_extractor_model.to(device)

  import numpy as np
  ##############
  #train set to vects
  ##############
  temp_ft, temp_lb = [], []
  i = 0
  for dt, lb in train_loader:
    if i % 500 == 0: print(f"Extracting batch {i} from train_loader.")
    i+=1
    tempfts = feature_extractor_model(dt.to(device))
    for j in range(tempfts.shape[0]):
      temp_ft.append(tempfts[j].reshape(-1).cpu().detach().numpy())
      temp_lb.append(lb[j].detach().numpy())

  train_ft = np.array(temp_ft)
  train_lb = np.array(temp_lb)
  ##############
  #test set to vects
  ##############
  temp_ft, temp_lb = [], []
  i = 0
  for dt, lb in test_loader:
    if i % 500 == 0: print(f"Extracting batch {i} from test_loader.")
    i+=1
    tempfts = feature_extractor_model(dt.to(device))
    for j in range(tempfts.shape[0]):
      temp_ft.append(tempfts[j].reshape(-1).cpu().detach().numpy())
      temp_lb.append(lb[j].detach().numpy())

  test_ft = np.array(temp_ft)
  test_lb = np.array(temp_lb)

  print()
  print(f"train_ft shape: {train_ft.shape}")
  print(f"train_lb shape: {train_lb.shape}")
  print(f"test_ft shape:  {test_ft.shape}")
  print(f"test_ft shape:  {test_lb.shape}")
  print(f"Feature extraction from {mdltype} (pretrained={pretrained}), is complete.")
  print()

  return train_ft, train_lb, test_ft, test_lb

In [4]:
def K9_stratified_class_sample(data_ft, data_lb, samp_per_cls=500, random_seed = False):
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  # COMP6248 Team K9 Density Estimators stratified sampler
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  # Author: Ian
  # Reviewed by: Nic
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  import numpy as np
  # clss = np.unique(data_lb)
  # ncls = len(clss)
  # samp_per_cls = 5000
  print(f"Choosing {samp_per_cls} samples per class.  data_lb has {len(np.unique(data_lb))} classes.")

  idxx = np.array([],dtype="int")
  for cls in np.unique(data_lb):
    idxx = np.append(idxx,
                    np.random.choice(np.where(data_lb == cls)[0],size=samp_per_cls))

  data_ft_sampled, data_lb_sampled = data_ft[idxx,:], data_lb[idxx]

  #check
  print("Summary of data_lb_sampled class sample size:")
  i = 1
  for cls in np.unique(data_lb):
    if i % 10 != 0:
      print(f"Cls {cls}: {np.sum(data_lb_sampled == cls)}",end= " | ")
    else:
      print(f"Cls {cls}: {np.sum(data_lb_sampled == cls)}")
    i += 1
  
  return data_ft_sampled, data_lb_sampled

In [5]:
def K9_OCSVM_v4(X_train, y_train, X_test, y_test, kernel_type='rbf', balance_testinoutlier=False,
                gammasklearndefault=False,specificgamma=None,papergammanumerator=10.0):
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  # COMP6248 Team K9 Density Estimators reproduction of OC-SVM logic based on paper's train.py specification
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  # Input: features and labels for train and test sets;
  #        kernel_type e{'rbf','linear'}
  #        balance_test_inoutlier: if True, inlier and outlier test sets are balanced by stratified sampling of outliers
  #        gammasklearndefault: if True, for RBF OC-SVM, sklearn default gamma is used;
  #        specificgamma: if None, the formula from the paper is used.  Else, the specific gamma to be used may be specified.
  #        papergammanumerator: 10.0 by default, the value of the numerator of the gamma function in the paper.  Setting to 1.0 would
  #                               yield the same value as sklearn default.  8.0 has been shown to yield results of interest for some runs.
  # Outputs: 1D array of class AUCs
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  # Author: Ian
  # Reviewed by: Marios
  ########################################################
  #Paper code: excerpts from train.py
  ########################################################
  # RBF kernel OC-SVM.
  #         feats_tr = tf.nn.l2_normalize(feats_tr, axis=1)
  #         feats = tf.nn.l2_normalize(feats, axis=1)
  #         # 10 times larger value of gamma.
  #         gamma = 10. / (tf.math.reduce_variance(feats_tr) * feats_tr.shape[1])
  #         clf = OneClassSVM(kernel='rbf', gamma=gamma).fit(feats_tr)
  #         scores = -clf.score_samples(feats)
  #         self.eval_metrics[key] = util_metric.roc(pr=scores, gt=labels)
  # Linear OC-SVM.
  #       elif 'locsvm' in key and key.startswith(('embed', 'pool')):
  #         # Linear kernel OC-SVM.
  #         clf = OneClassSVM(kernel='linear').fit(feats_tr)
  #         scores = -clf.score_samples(feats)
  #         self.eval_metrics[key] = util_metric.roc(pr=scores, gt=labels)
  ########################################################
  #Marios review of Ian code 06/05/21:
  ########################################################
  ## 1) try tf.nn.l2_normalise vs sklearn normalise.  Ian: tested on f-MNIST+checked the documentation; got exactly the same results, pretty sure they are the same.
  ## 2) double check the -1 or 0 business  - does it make a difference? Ian: tested, makes no difference; documentation states it works off the class with 'the greater label'.

  import sklearn
  from sklearn.pipeline import make_pipeline
  from sklearn.preprocessing import normalize
  from sklearn.svm import OneClassSVM
  from sklearn import metrics
  from sklearn.metrics import roc_auc_score, roc_curve, auc

  print("Starting K9 OC-SVM, kernel_type = "+kernel_type)
  print(f"X_train.shape: {X_train.shape}, y_train.shape: {y_train.shape}")
  print(f"X_test.shape: {X_test.shape}, y_test.shape: {y_test.shape}")
  print()
  train_classes, per_class_auc, one_point_per_class_auc = np.unique(y_train), [], []
  for one_class in train_classes:
    #one_class = train_classes[1]
    #Normalise train set; set gamma (both as per the paper)
    OC_X_train = X_train[y_train==one_class,:]
    OC_X_train_normalised = normalize(OC_X_train,norm='l2',axis=1)   #normalise by L2 norm on a **row** basis
    
    #Fit the OC-SVM
    print(f"np.var(OC_X_train_normalised):{np.var(OC_X_train_normalised)}")
    if kernel_type == 'rbf':
      if gammasklearndefault:
        clf = OneClassSVM(kernel='rbf')
        print(f"gamma: as per sklearn default.")
      else:
        if specificgamma == None:
          gamma = papergammanumerator/(np.var(OC_X_train_normalised) * OC_X_train_normalised.shape[1])
          #gamma = papergammanumerator/(np.var(OC_X_train) * OC_X_train_normalised.shape[1])
          print(f"gamma: as per paper: {gamma}")
        else:
          gamma = specificgamma
          print(f"gamma: specified by user: {gamma}")
        clf = OneClassSVM(kernel='rbf', gamma=gamma)
    else:
      clf = OneClassSVM(kernel='linear')
    clf = clf.fit(OC_X_train_normalised)

    #Balance data, if specified
    if balance_testinoutlier == True:
      idxx = np.where(y_test == one_class)[0]
      idxx = np.append(idxx,
                np.random.choice(np.where(y_test != one_class)[0],
                        size=len(idxx),
                        replace=False))
      X_test_cls = X_test[idxx]
      y_test_cls = y_test[idxx]
    else:
      X_test_cls = X_test
      y_test_cls = y_test

    #Use fitted model to make predictions on test
    X_test_normalised = normalize(X_test_cls,norm='l2',axis=1)   #normalise by L2 norm on a **row** basis
    y_test_pred = clf.predict(X_test_normalised)
    y_test_pred_scores = clf.score_samples(X_test_normalised)
    y_test_pred_AUC = roc_auc_score(1.*(y_test_cls==one_class),y_test_pred_scores)

    #save AUC
    per_class_auc = np.append(per_class_auc, y_test_pred_AUC)

    #Print out results
    y_test_tp, y_test_tn = np.sum(y_test_pred[y_test_cls==one_class] == 1), np.sum(y_test_pred[y_test_cls!=one_class] == -1)
    y_test_fp, y_test_fn = np.sum(y_test_pred[y_test_cls!=one_class] == 1), np.sum(y_test_pred[y_test_cls==one_class] == -1)
    y_test_n_inlier, y_test_n_outlier = np.sum(y_test_cls==one_class), np.sum(y_test_cls!=one_class)
    y_test_fpr, y_test_tpr = y_test_fp/(y_test_fp+y_test_tn), y_test_tp/(y_test_tp+y_test_fn)
    y_test_one_point_auc = 0.5*(1-y_test_fpr+y_test_tpr)
    #save one_point_auc
    one_point_per_class_auc = np.append(one_point_per_class_auc, y_test_one_point_auc)
    print(f"Class: {one_class}")
    print(f"AUC score: {y_test_pred_AUC: .4f}, Accuracy: {(y_test_tp+y_test_tn)/len(y_test_cls): .4f}")
    print(f"OC_X_train shape: {OC_X_train.shape}, X_test_cls shape: {X_test_cls.shape}.")
    print(f"Results after model application to *test* set: n_inlier: {y_test_n_inlier} ; n_outlier: {y_test_n_outlier}")
    print(f"TP: {y_test_tp}, TN: {y_test_tn}, FP: {y_test_fp}, FN: {y_test_fn}")
    print(f"TPR: {y_test_tpr}, FPR: {y_test_fpr}, one-point AUC:{y_test_one_point_auc}")
    print()
  print(f"Unweighted mean AUC: {np.mean(per_class_auc): .4f}")
  print(f"Unweighted one-point mean AUC: {np.mean(one_point_per_class_auc): .4f}")
  return per_class_auc

In [6]:
def K9_pd_runsummary():
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  # COMP6248 Team K9 Density Estimators summary of run configuration
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  # Author: Ian
  # Reviewed by: Nic
  #*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
  import pandas as pd
  run_params  = pd.DataFrame([dataset, batch_size, mdltype, pretrained, samp_per_cls, random_seed, kernel_type,
                        X_train.shape[0], X_train.shape[1], y_train.shape[0], X_test.shape[0], X_test.shape[1], y_test.shape[0],np.mean(res)])
  run_results = pd.DataFrame(res)
  run_summary = pd.concat([run_params,run_results],ignore_index=True)
  return run_summary

#Run One Configuration

Minimal example of loading and running one run

In [12]:
#Minimal example of loading and running one run
train_loader, test_loader = K9_dataloader(dataset="CIFAR10", batch_size=128)
train_ft, train_lb, test_ft, test_lb = K9_resnet1850_ftex(train_loader, test_loader, mdltype="resnet18", pretrained=False)
X_train, y_train = K9_stratified_class_sample(train_ft, train_lb, samp_per_cls=5000)
X_test, y_test   = test_ft, test_lb
res = K9_OCSVM_v4(X_train, y_train, X_test, y_test, kernel_type='rbf', balance_testinoutlier=True)

Files already downloaded and verified
Files already downloaded and verified

train_loader batches: 391, test_loader batches: 79, batch_size: 128
train_shuffle set to False
train_set length: 50000, test_set length: 10000
train_loader and test_loader ready for CIFAR10.

Thu May 13 08:04:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    34W / 250W |   4093MiB / 16280MiB |   

#Run a Set of Configurations

The minimal example was easily extended to reproduce all results in from one cell, saving outputs to Google Drive.
One run reproduction run will take about 1.5hr to run on Colab Pro with a P100; making five runs takes 7hr.  Code depends on your Google Drive file paths being specified correctly.  CSV of each run's results is saved, along with a second CSV with a rolling summary of runs.

In [8]:
# import time
# from google.colab import drive
# drive.mount('/content/gdrive')

# reproductions = 5
# datasets = ["fMNIST", "CIFAR10", "CIFAR100"]
# batch_size = 16
# samp_per_cls = 500
# random_seed = False
# mdltypes = ["resnet18","resnet50"]
# save_runsummary = True
# save_runsummary_collection = True

# runsummary_collection = pd.DataFrame([])
# for reproduction in range(reproductions):
#   print(f"Beginning reproduction {reproduction} of {reproductions}.")
#   for dataset in datasets:
#     train_loader, test_loader = K9_dataloader(dataset=dataset,
#                                               batch_size=batch_size)   #LOAD the data
#     for mdltype in mdltypes:
#       pretrained = True if mdltype == "resnet50" else False   #resnet50 pretrained, resnet18 random weights/untrained
#       train_ft, train_lb, test_ft, test_lb = K9_resnet1850_ftex(train_loader,test_loader,mdltype=mdltype,pretrained=pretrained) #EXTRACT the features
#       X_train, y_train = K9_stratified_class_sample(train_ft,train_lb,samp_per_cls=samp_per_cls,random_seed=random_seed) #REDUCE the sample size by stratified sampling
#       X_test, y_test = test_ft, test_lb #KEEP test set the same size

#       for kernel_type in ["linear","rbf"]:
#         res = K9_OCSVM_v4(X_train, y_train, X_test, y_test, kernel_type)  #Apply OC-SVM
        
#         run_summary = K9_pd_runsummary()  #Summarise the run
#         print(run_summary)
#         if save_runsummary: run_summary.to_csv("/content/gdrive/MyDrive/COMP6248CW/run_summary_"+time.strftime("%Y%m%d-%H%M%S")+".csv")

#         runsummary_collection = pd.concat([runsummary_collection,run_summary], axis = 1) #Append as a new column in runsummary
#         if save_runsummary_collection: runsummary_collection.to_csv("/content/gdrive/MyDrive/COMP6248CW/run_summary_collection"+time.strftime("%Y%m%d-%H%M%S")+".csv")